In [2]:
from typing import List, Dict, cast, Set
import os
from matplotlib.pyplot import text
from natsort import natsorted
import numpy as np

import hydra
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer
import quartz

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import qtz

import plotly.graph_objects as go
import plotly.express as px
from IPython import embed
from icecream import ic

qtz.init_quartz_context(
    ['h', 'cx', 'x', 'rz', 'add',],
    '../../ecc_set/nam_ecc.json',
    False,
    True,
)
ic(qtz.quartz_context.num_xfers)

/home/jinjun/miniconda3/envs/quartz/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ic| qtz.quartz_context.num_xfers: 8669


8669

In [3]:
def y_to_step_from_seq(dir_path: str, seq_name: str) -> List[np.ndarray]:
    """
    Return:
      [ [x_0, x_1, ...], [y_0, y_1, ...] ]
    """
    X_s: List[int] = []
    Y_s: List[int] = []
    xfers_used: Set[int] = set()
    inc_xfers_used: Set[int] = set()

    def one_stage(inner_dir_path: str, base_step: str) -> int:
        circ_files: List[str] = natsorted(os.listdir(inner_dir_path))
        graph_list: List[quartz.PyGraph] = []
        for circ_file in circ_files:
            # 10_54_47_8323.qasm
            splitted_fname = os.path.splitext(circ_file)[0].split('_')
            if len(splitted_fname) != 4:
                print(f'Warning: ignore {circ_file} !', file=sys.stderr)
                continue
            i_step, cost, action_node, action_xfer = list(map(int, splitted_fname))
            
            if i_step == 0 and base_step > 0:
              continue
            i_step += base_step
            X_s.append(i_step)
            Y_s.append(cost)
            if i_step == 0:
                continue
            xfers_used.add(action_xfer)
            if cost > Y_s[-2]:
                inc_xfers_used.add(action_xfer)
        # end for
        return i_step
    # end def
    
    if os.path.isfile(
        os.path.join(dir_path, os.listdir(dir_path)[0])
    ):
        one_stage(dir_path, base_step=0)
    else:
        inner_dirs = natsorted(os.listdir(dir_path), reverse=True)
        last_step: int = 0
        for inner_dir in inner_dirs:
            full_inner_dir_path = os.path.join(dir_path, inner_dir)
            last_step = one_stage(full_inner_dir_path, last_step)

    ic(seq_name, len(xfers_used), len(inc_xfers_used))

    X = np.array(X_s)
    Y = np.array(Y_s)

    # scaled num to reduce
    # tot_reduced = Y_s[0] - Y_s[-1]
    # Y = (Y - Y_s[-1]) / tot_reduced
    
    return [X, Y]



In [3]:
def draw_multi_seq(
    seqs_dir_path: str = 'full_seq',
) -> go.Figure:
    seq_dirs: List[str] = natsorted(os.listdir(seqs_dir_path))

    fig = go.Figure()

    for seq_dir in seq_dirs:
        seq_name = seq_dir.split('_fullseq')[0]
        full_seq_path = os.path.join(seqs_dir_path, seq_dir)
        if not os.path.isdir(full_seq_path):
            print(f'Warning: ignore {full_seq_path} since it\'s not a dir.', file=sys.stderr)
            continue
        # if 'gf2^8' not in seq_name:
        #     continue
        X_s, Y_s = y_to_step_from_seq(
            full_seq_path,
            seq_name,
        )

        fig.add_trace(go.Scatter(
            x=X_s, y=Y_s,
            mode='lines+markers',
            name=f'{seq_name}',
            marker_size=1,
        ))
    # end for
    fig.update_layout(
        title={
            'text': f'Optimization Sequences',
            'x': 0.5,
        },
        xaxis_title=f'# steps',
        yaxis_title=f'gates', # f'% gates remained to opt. to achieve SOTA',
    )
    return fig

In [8]:
def draw_multi_seq_group(
    seqs_dir_path: str = 'small_seqs',
    circs: List[str] = None,
    scale: bool = False,
) -> go.Figure:
    seq_dirs: List[str] = natsorted(os.listdir(seqs_dir_path))

    fig = go.Figure()
    color_seq = px.colors.qualitative.Plotly

    for i_circ, circ in enumerate(circs):
        inc_seq_dir = os.path.join(seqs_dir_path, f'{circ}-inc')
        noinc_seq_dir = os.path.join(seqs_dir_path, f'{circ}-noinc')

        X_inc, Y_inc = y_to_step_from_seq(
            inc_seq_dir,
            f'{circ}-inc',
        )
        X_noinc, Y_noinc = y_to_step_from_seq(
            noinc_seq_dir,
            f'{circ}-noinc',
        )
        yaxis_title = '#gates'
        if scale:
            factor = Y_inc[0]
            Y_inc = Y_inc / factor
            Y_noinc = Y_noinc / factor
            yaxis_title = '#gates / init #gates'
        # ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
        fig.add_traces([
            go.Scatter(
                x=X_inc, y=Y_inc, mode='lines',
                line=dict(color=color_seq[i_circ], dash='solid'),
                legendgroup=circ, legendgrouptitle_text=circ,
                name='w/ inc',
            ),
            go.Scatter(
                x=X_noinc, y=Y_noinc, mode='lines',
                line=dict(color=color_seq[i_circ], dash='dot'),
                legendgroup=circ, legendgrouptitle_text=circ,
                name='w/o inc',
            ),
        ])
    # end for
    fig.update_layout(
        title={
            'text': f'Sequences of Optimization',
            'x': 0.5,
        },
        xaxis_title=f'length of sequence',
        yaxis_title=yaxis_title,
    )
    fig.update_layout(
        legend=dict(
            yanchor="top", y=0.99,
            xanchor="right", x=0.99,
        ),
        # margin=dict(l=0,r=0,b=0,t=0),
        font=dict(size=24, color='black'),
    )
    # fig.update_xaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_yaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    # })
    return fig

fig = draw_multi_seq_group(
    seqs_dir_path='small_seqs',
    scale=True,
    circs=[ 'barenco_tof_3', 'mod5_4' ]
)
fig.show()
fig.write_html(f'seq.html')
fig.write_image(f'opt_seq.pdf', width=600, height=500)

ic| seq_name: 'barenco_tof_3-inc'
    len(xfers_used): 172
    len(inc_xfers_used): 52
ic| seq_name: 'barenco_tof_3-noinc'
    len(xfers_used): 22
    len(inc_xfers_used): 0
ic| seq_name: 'mod5_4-inc'
    len(xfers_used): 143
    len(inc_xfers_used): 30
ic| seq_name: 'mod5_4-noinc'
    len(xfers_used): 81
    len(inc_xfers_used): 0
